In [1]:
# ! pip install flipside

In [1]:
from dotenv import load_dotenv
import inspect
import os
import pandas as pd

from utils import init_flipside, execute_query

from queries import (
    bridge,
    dex,
    farcaster,
    lending,
    marketplace,
    nft,
    rwa,
    staking
)

In [2]:
load_dotenv()
FLIPSIDE_API_KEY = os.getenv('FLIPSIDE_API_KEY')
flipside = init_flipside(FLIPSIDE_API_KEY)
apps = pd.read_csv('data/applications_reviewed.csv', index_col=0)

In [3]:
METRIC_MODULES = {
    'Art NFTs': {
        "module": nft,
        "params": ["address"]
    },
    'Other Media NFTs': {
        "module": nft,
        "params": ["address"]
    },
    'Bridges': {
        "module": bridge,
        "params": ["chain", "address"]
    },
    'DEX': {
        "module": dex,
        "params": ["chain", "address"]
    },
    'Channels': {
        "module": farcaster,
        "params": ["warpcast_url"]
    },
    'Frames': {
        "module": None, # needs to be updated
        "params": ["warpcast_url"]
    },
    'Lending': {
        "module": lending,
        "params": ["chain", "address"]
    },
    'Art Marketplace': {
        "module": marketplace,
        "params": ["chain", "address"]
    },
    'Other Media Marketplace': {
        "module": marketplace,
        "params": ["chain", "address"]
    },
    'Real World Assets': {
        "module": rwa,
        "params": ["chain", "address"]
    },
    'Staking': {
        "module": staking,
        "params": ["chain", "address"]
    },
    'Community & Curation': {
        "module": None,
        "params": ["chain", "address"]
    },
    'Gaming': {
        "module": None,
        "params": ["chain", "address"]
    },
    'Airdrop': {
        "module": None,
        "params": ["chain", "address"]
    },
    'Meme Community': {
        "module": None,
        "params": ["chain", "address"]
    },
    'Payments': {
        "module": None,
        "params": ["chain", "address"]
    },    
    'Donations': {
        "module": None,
        "params": ["chain", "address"]
    },
    'Identity': {
        "module": None,
        "params": ["chain", "address"]
    },
    'Betting & Prediction Markets': {
        "module": None,
        "params": ["chain", "address"]
    },
}

In [4]:
def query_project_metrics(project_data):    
    uid = project_data['id']
    category = project_data['category']
    module_name = METRIC_MODULES.get(category).get('module')
    if not module_name:
        return
    
    metric_modules = inspect.getmembers(module_name, inspect.isfunction)
    expected_params = METRIC_MODULES.get(category).get('params', [])
    params = [project_data.get(p) for p in expected_params]
    if len(params) != len(expected_params):
        print("Unable to find params:", expected_params)
        return
    
    metrics = []
    project_info = {'id': uid, 'category': category}
    project_info.update(dict(zip(expected_params, params)))
    for (query_name, query_fn) in metric_modules:
        sql = query_fn(*params)
        value = execute_query(flipside, sql, project_info)
        metric_info = project_info.copy()
        metric_info.update({'query_name': query_name, 'value': value})
        metrics.append(metric_info)
    return metrics

In [6]:
results = query_project_metrics(apps.iloc[1])
results

TypeError: artnft_nam() missing 1 required positional argument: 'wallet_address'

In [7]:
apps.iloc[1]

id                        0x5a587244ecf18246e881cda06bac1ece7c0995bb2faf...
recipient                        0x0000000000000000000000000000000000000000
time                                                             1724723573
name                                                           1001 raisons
profile_name                                                           kawz
profile_url                                       https://warpcast.com/kawz
metadata_name                                                  1001 raisons
metadata_website                       https://www.sound.xyz/medmc/releases
project_type                                                        Creator
category                                                           Art NFTs
address_type                                                  mintingWallet
address                          0x8f59aa0f586d6d941152b9075b344cdc42e9b024
chain_id                                                                NaN
chain       